In [23]:
import sys
import os 


print(sys.path)


import QUANTAXIS as QA
from QUANTAXIS.QAAnalysis.QAAnalysis_finance import QAAnalysis_finance
from QUANTAXIS.QASetting.QALocalize import analysis_path
from tools import Sample_Tools as smpl
from tools import QAAdapter as qaadapter
# from tools import QAAdapter as qaadapter
# from tools import QA_adapter_get_code_from_block
from base.Constants import trade_date_sse

block_name = '通信设备'
block_name = '旅游景点'
block_name = '影视音像'
block_name = '电器仪表'
block_name = '软件服务'

gap = 90
end = '2023-06-21'
start = trade_date_sse[trade_date_sse.index(end)-gap]

# a  = qaadapter.QA_adapter_get_code_from_block('sw_l2', '水泥')
stock_list  = qaadapter.QA_adapter_get_code_from_block('sw_l1', '国防军工')

print(smpl.get_stock_name(stock_list), len(stock_list))

from ind.Alpha101 import *

data = QA.QA_fetch_stock_day_adv(stock_list, start, end)
# data.data.index = data.data.index.map(lambda x: (x[0].tz_localize('UTC'), x[1]))
# data.data.index.names = ['tz', 'code']
cur_ret = smpl.get_current_return(data,'close')
prices = data.data['close']

factor = alpha1(prices, cur_ret)
factor = factor.replace([-np.inf, np.inf], 0).fillna(value=0)

prices = prices.unstack()

# formater = '%Y%m%d' # '%Y%m%d %H%M%S'
# index_ = [pd.to_datetime(x).strftime(formater) for x in inds_.index.values]
# factor = alpha9(price)
print(factor.tail(10))
print(prices.tail(10))

['/home/disk1/quantaxis/kaikai/IndicatorLab', '/home/work_space/IndicatorLab/factor_data', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/opt/quant/lib/python3.8/site-packages', '/home/disk1/data_framework/pytdx', '/home/disk1/data_framework/akshare', '/home/disk1/data_framework/efinance', '/home/work_space/QUANTAXIS_Learning', '/home/disk1/quanttopian/alphalens', '/home/disk1/quanttopian/empyrical', '/home/disk1/quanttopian/pyfolio', '/home/disk1/quantaxis/kaikai', '/home/disk1/quantaxis/kaikai/IndicatorLab', '/home/work_space/IndicatorLab/tools/', '/home/work_space/IndicatorLab/tools/', '/home/work_space/IndicatorLab/', '/home/work_space/IndicatorLab/tools/', '/home/work_space/IndicatorLab/tools/']
code
000519     中兵红箭
000547     航天发展
000561     烽火电子
000576     甘化科工
000697    *ST炼石
          ...  
603261     立航科技
603267     鸿远电子
603678     火炬电子
603712      七一二
605123     派克新材
Name: name, Length: 61, dtype: object 61
date        code  
2023-06-2

In [ ]:

from tools.factorBase import FactorBase

factor_base = FactorBase("alpha101_1", host=clickhouse_ip, port=clickhouse_port, user=clickhouse_user, password=clickhouse_password):


In [ ]:
from clickhouse_driver import Client
import clickhouse_driver

import QUANTAXIS as QA
import pandas as pd
port = 9001
host = 'localhost'
user = 'admin'
password = 'admin'
database = 'test'

#  database=database
client = clickhouse_driver.Client(host=host, port=port, user=user, database=database, password=password,
                                               settings={
                                                   'insert_block_size': 100000000},
                                               compression=True)
# 创建 ClickHouse 客户端连接

client.execute('CREATE DATABASE IF NOT EXISTS test')

# 创建表
client.execute('CREATE TABLE IF NOT EXISTS test_table (id Int, name String) ENGINE = MergeTree() ORDER BY id')

# 插入数据
client.execute('INSERT INTO test_table (id, name) VALUES', [(1, 'John'), (2, 'Alice'), (3, 'Bob')])

# 执行查询并获取结果
result = client.execute('SELECT * FROM test_table')

# 打印查询结果
for row in result:
    print(row)

# 更新数据
# client.execute('ALTER TABLE test_table UPDATE name = "HELLO" WHERE id = 2')
client.execute('ALTER TABLE test_table UPDATE name = \'HELLO\' WHERE id = 2')


# change = 'ALTER TABLE test_table MODIFY COLUMN id Int32;\
# \
# DELETE FROM test_table\
# WHERE id IN (\
#     SELECT id\
#     FROM (\
#         SELECT id, ROW_NUMBER() OVER(PARTITION BY id ORDER BY id) AS row_num\
#         FROM test_table\
#     ) AS subquery\
#     WHERE row_num > 1\
# );'

# client.execute(change)

change = 'ALTER TABLE test_table MODIFY COLUMN id Int32;\
\
DELETE FROM test_table\
WHERE id IN (\
    SELECT id\
    FROM (\
        SELECT id, ROW_NUMBER() OVER(PARTITION BY id ORDER BY id) AS row_num\
        FROM test_table\
    ) AS subquery\
    WHERE row_num > 1\
);'

client.execute(change)

# # client.execute("INSERT INTO test_table (id, name) VALUES (2, 'Mike')" )
result = client.execute('SELECT * FROM test_table')

# 打印查询结果
for row in result:
    print(row)
# # 删除数据
# client.execute('DELETE FROM test_table WHERE id = 3')

# 删除表
# client.execute('DROP TABLE IF EXISTS test_table')